In [1]:
#Source for map tuning code: https://medium.com/stereopi/opencv-and-depth-map-on-stereopi-tutorial-62cb6792bbed
%matplotlib widget
import cv2
import os
from matplotlib import pyplot as plt
from matplotlib.widgets import Slider, Button
import numpy as np
import json
from stereovision.calibration import StereoCalibrator
from stereovision.calibration import StereoCalibration
import ipywidgets as wg
from IPython.display import display
import seaborn as sns                  

In [2]:
photo_width = 1280
photo_height = 480
image_height = photo_height
image_width = int(photo_width/2)
def getImagePath(width, height, num):
    return './scenes2/240in/scene_'+str(width)+'x'+str(height)+'_'+str(num)+'.png'

# def setResolution(width, height):
#     global photo_width, photo_height, image_height, image_width
#     photo_width = 1280
#     photo_height = 480
#     image_height = photo_height
#     image_width = int(photo_width/2)

In [3]:
fig, ax = plt.subplots()
image_size = (image_width, image_height)
imageToDisp = getImagePath(photo_width, photo_height, 1)
if os.path.isfile(imageToDisp) == False:
    print('Can not read image from file \"' + imageToDisp + '\"')
    exit(0)

pair_img = cv2.imread(imageToDisp, 0)
print('Read and split image...')
imgLeft = pair_img[0:photo_height, 0:image_width]  # Y+H and X+W
imgRight = pair_img[0:photo_height, image_width:photo_width]  # Y+H and X+W

# Implementing calibration data
print('Read calibration data and rectifying stereo pair...')
calibration = StereoCalibration(input_folder='calib_result2')
rectified_pair = calibration.rectify((imgLeft, imgRight))
focal_length = -1*calibration.disp_to_depth_mat[2][3]

ax.imshow(rectified_pair[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Can not read image from file "./scenes2/240in/scene_1280x480_1.png"
Read and split image...


TypeError: 'NoneType' object is not subscriptable

In [ ]:
# Depth map function
SWS = 5
PFS = 5
PFC = 29
MDS = -25
NOD = 128
TTH = 100
UR = 10
SR = 15
SPWS = 100

def stereo_depth_map(rectified_pair):
    print('SWS=' + str(SWS) + ' PFS=' + str(PFS) + ' PFC=' + str(PFC) + ' MDS=' + \
          str(MDS) + ' NOD=' + str(NOD) + ' TTH=' + str(TTH))
    print(' UR=' + str(UR) + ' SR=' + str(SR) + ' SPWS=' + str(SPWS))
    c, r = rectified_pair[0].shape
    disparity = np.zeros((c, r), np.uint8)
    sbm = cv2.StereoBM_create(numDisparities=16, blockSize=15)
    # sbm.SADWindowSize = SWS
    sbm.setPreFilterType(1)
    sbm.setPreFilterSize(PFS)
    sbm.setPreFilterCap(PFC)
    sbm.setMinDisparity(MDS)
    sbm.setNumDisparities(NOD)
    sbm.setTextureThreshold(TTH)
    sbm.setUniquenessRatio(UR)
    sbm.setSpeckleRange(SR)
    sbm.setSpeckleWindowSize(SPWS)
    dmLeft = rectified_pair[0]
    dmRight = rectified_pair[1]
    # cv2.FindStereoCorrespondenceBM(dmLeft, dmRight, disparity, sbm)
    disparity = sbm.compute(dmLeft, dmRight)
    # disparity_visual = cv.CreateMat(c, r, cv.CV_8U)
    local_max = disparity.max()
    local_min = disparity.min()
    print("MAX " + str(local_max))
    print("MIN " + str(local_min))
    disparity_visual = (disparity - local_min) * (1.0 / (local_max - local_min))
    local_max = disparity_visual.max()
    local_min = disparity_visual.min()
    print("MAX " + str(local_max))
    print("MIN " + str(local_min))
    # cv.Normalize(disparity, disparity_visual, 0, 255, cv.CV_MINMAX)
    # disparity_visual = np.array(disparity_visual)
    return disparity_visual
disparity = stereo_depth_map(rectified_pair)

In [ ]:
# Set up and draw interface
# Draw left image and depth map
axcolor = 'lightgoldenrodyellow'
fig = plt.subplots(1, 2)
plt.subplots_adjust(left=0.15, bottom=0.5)
plt.subplot(1, 2, 1)
dmObject = plt.imshow(rectified_pair[0], 'gray')

saveax = plt.axes([0.3, 0.38, 0.15, 0.04])  # stepX stepY width height
buttons = Button(saveax, 'Save settings', color=axcolor, hovercolor='0.975')
def save_map_settings(event):
    buttons.label.set_text("Saving...")
    print('Saving to file...')
    result = json.dumps({'SADWindowSize': SWS, 'preFilterSize': PFS, 'preFilterCap': PFC, \
                         'minDisparity': MDS, 'numberOfDisparities': NOD, 'textureThreshold': TTH, \
                         'uniquenessRatio': UR, 'speckleRange': SR, 'speckleWindowSize': SPWS}, \
                        sort_keys=True, indent=4, separators=(',', ':'))
    fName = '3dmap_set.txt'
    f = open(str(fName), 'w')
    f.write(result)
    f.close()
    buttons.label.set_text("Save to file")
    print('Settings saved to file ' + fName)


buttons.on_clicked(save_map_settings)
loadax = plt.axes([0.5, 0.38, 0.15, 0.04])  # stepX stepY width height
buttonl = Button(loadax, 'Load settings', color=axcolor, hovercolor='0.975')

def load_map_settings(event):
    global SWS, PFS, PFC, MDS, NOD, TTH, UR, SR, SPWS, loading_settings
    loading_settings = 1
    fName = '3dmap_set.txt'
    print('Loading parameters from file...')
    buttonl.label.set_text("Loading...")
    f = open(fName, 'r')
    data = json.load(f)
    sSWS.set_val(data['SADWindowSize'])
    sPFS.set_val(data['preFilterSize'])
    sPFC.set_val(data['preFilterCap'])
    sMDS.set_val(data['minDisparity'])
    sNOD.set_val(data['numberOfDisparities'])
    sTTH.set_val(data['textureThreshold'])
    sUR.set_val(data['uniquenessRatio'])
    sSR.set_val(data['speckleRange'])
    sSPWS.set_val(data['speckleWindowSize'])
    f.close()
    buttonl.label.set_text("Load settings")
    print('Parameters loaded from file ' + fName)
    print('Redrawing depth map with loaded parameters...')
    loading_settings = 0
    update(0)
    print('Done!')

buttonl.on_clicked(load_map_settings)

plt.subplot(1, 2, 2)
dmObject = plt.imshow(disparity, aspect='equal', cmap='jet')

# Draw interface for adjusting parameters
print('Start interface creation (it takes up to 30 seconds)...')

SWSaxe = plt.axes([0.15, 0.01, 0.7, 0.025])  # , axisbg=axcolor) #stepX stepY width height
PFSaxe = plt.axes([0.15, 0.05, 0.7, 0.025])  # , axisbg=axcolor) #stepX stepY width height
PFCaxe = plt.axes([0.15, 0.09, 0.7, 0.025])  # , axisbg=axcolor) #stepX stepY width height
MDSaxe = plt.axes([0.15, 0.13, 0.7, 0.025])  # , axisbg=axcolor) #stepX stepY width height
NODaxe = plt.axes([0.15, 0.17, 0.7, 0.025])  # , axisbg=axcolor) #stepX stepY width height
TTHaxe = plt.axes([0.15, 0.21, 0.7, 0.025])  # , axisbg=axcolor) #stepX stepY width height
URaxe = plt.axes([0.15, 0.25, 0.7, 0.025])  # , axisbg=axcolor) #stepX stepY width height
SRaxe = plt.axes([0.15, 0.29, 0.7, 0.025])  # , axisbg=axcolor) #stepX stepY width height
SPWSaxe = plt.axes([0.15, 0.33, 0.7, 0.025])  # , axisbg=axcolor) #stepX stepY width height

sSWS = Slider(SWSaxe, 'SWS', 5.0, 255.0, valinit=5)
sPFS = Slider(PFSaxe, 'PFS', 5.0, 255.0, valinit=5)
sPFC = Slider(PFCaxe, 'PreFiltCap', 5.0, 63.0, valinit=29)
sMDS = Slider(MDSaxe, 'MinDISP', -100.0, 100.0, valinit=-25)
sNOD = Slider(NODaxe, 'NumOfDisp', 16.0, 256.0, valinit=128)
sTTH = Slider(TTHaxe, 'TxtrThrshld', 0.0, 1000.0, valinit=100)
sUR = Slider(URaxe, 'UnicRatio', 1.0, 20.0, valinit=10)
sSR = Slider(SRaxe, 'SpcklRng', 0.0, 40.0, valinit=15)
sSPWS = Slider(SPWSaxe, 'SpklWinSze', 0.0, 300.0, valinit=100)

# Update depth map parameters and redraw
def update(val):
    global SWS, PFS, PFC, MDS, NOD, TTH, UR, SR, SPWS
    SWS = int(sSWS.val / 2) * 2 + 1  # convert to ODD
    PFS = int(sPFS.val / 2) * 2 + 1
    PFC = int(sPFC.val / 2) * 2 + 1
    MDS = int(sMDS.val)
    NOD = int(sNOD.val / 16) * 16
    TTH = int(sTTH.val)
    UR = int(sUR.val)
    SR = int(sSR.val)
    SPWS = int(sSPWS.val)
    if (loading_settings == 0):
        print('Rebuilding depth map')
        disparity = stereo_depth_map(rectified_pair)
        dmObject.set_data(disparity)
        print('Redraw depth map')
        plt.draw()

# Connect update actions to control elements
sSWS.on_changed(update)
sPFS.on_changed(update)
sPFC.on_changed(update)
sMDS.on_changed(update)
sNOD.on_changed(update)
sTTH.on_changed(update)
sUR.on_changed(update)
sSR.on_changed(update)
sSPWS.on_changed(update)

print('Show interface to user')
plt.show()

In [ ]:
def generateBinaryImg(I):
    I_intensity = (I[:,:,1] > 1000) # identify pixels with low GREEN values - they are less reliable
    I_cr = I[:,:,0]/I[:,:,1]  # This is the ratio of Red (0) to Green (1)
    I_cr[~I_intensity] = 1000    # small green values would have infinite or near infinite ratios, replace with a "big" number instead for visualization
    I_sig = (I[:,:,0] > 1.0*I[:,:,1]) & I_intensity # Create image identifying flames on brightness & color basis
    I_sig = np.uint8(I_sig)
    I_processed = cv2.morphologyEx(I_sig, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (4,4)))
    return I_processed

def distance_to_camera(avg_disparity):
    # compute and return the distance from the maker to the camera
    return (base_offset*focal_length)/avg_disparity

def fire_disparity(bin_img, disparity):
    return np.where(bin_img, disparity,0) #replace all fire pixels with disparity values


In [ ]:
def generateBinaryImgStereo(I):
    I_intensity = (I[:,:,1] > 100) # identify pixels with low GREEN values - they are less reliable
    I_cr = I[:,:,2]/I[:,:,1]  # This is the ratio of Red (0) to Green (1)
    I_cr[~I_intensity] = 1.0    # small green values would have infinite or near infinite ratios, replace with a "big" number instead for visualization
    I_sig = (I[:,:,2] > 1.2*I[:,:,1]) & I_intensity # Create image identifying flames on brightness & color basis
#     I_sig = (I[:,:,2]>220) & (I[:,:,2]<256)
#     I_sig = (I_cr>1.1)
    I_sig = np.uint8(I_sig)
    I_processed = cv2.morphologyEx(I_sig, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (4,4)))
    return I_processed

fig, ax = plt.subplots(ncols = 2)
imageToDisp = getImagePath(photo_width, photo_height, 1)
I = cv2.imread(imageToDisp)
imgLeft_RGB = I[0:photo_height, 0:image_width, :]
bin_img = generateBinaryImgStereo(imgLeft_RGB)
ax[0].imshow(imgLeft_RGB)
ax[1].imshow(bin_img)

In [ ]:
fig, ax = plt.subplots()
I = imgLeft_RGB
I_intensity = (I[:,:,1] > 100) # identify pixels with low GREEN values - they are less reliable
I_cr = I[:,:,2]/I[:,:,1]  # This is the ratio of Red (0) to Green (1)
I_cr[~I_intensity] = 100    # small green values would have infinite or near infinite ratios
sns.distplot(I_cr[I_intensity],ax= ax,hist = True,norm_hist = False, kde = False)
ax.set_ylabel("Number of Pixels")
ax.set_xlabel("Color Ratio: B/G")
ax.set_title("Distribution of B/G ratios ")
plt.show()

In [ ]:
fig, ax= plt.subplots() 
num_images = 5
for i in range(1,num_images+1):
    imageToDisp = getImagePath(photo_width, photo_height, i)
    I = cv2.imread(imageToDisp)
    I_intensity = (I[:,:,1] > 100) # identify pixels with low GREEN values - they are less reliable
    I_cr = I[:,:,2]/I[:,:,1]  # This is the ratio of Red (0) to Green (1)
    I_cr[~I_intensity] = 100    # small green values would have infinite or near infinite ratios, replace with a "big" number instead for visualization
    mean = np.mean(I_cr[I_intensity])
    plt.errorbar(np.mean(I_intensity), mean, np.std(I_cr[I_intensity]), linestyle='None', marker='o')

#plt.errorbar(imageNum, mean, np.std(I_cr[I_intensity]), linestyle='None', marker='o')
ax.set_ylabel("Color Ratio: R/G")
ax.set_xlabel("Mean Intensity")
ax.set_title("Error in Distribution of R/G Ratios")
plt.show()

In [ ]:
fig, ax = plt.subplots(ncols = 3)
I_cr = imgLeft_RGB[:,:,0]/imgLeft_RGB[:,:,1]
ax[0].imshow(imgLeft_RGB[:,:,0])
ax[1].imshow(imgLeft_RGB[:,:,1])
ax[2].imshow(imgLeft_RGB[:,:,2])
ax[0].set_title("Red")
ax[1].set_title("Green")
ax[2].set_title("Blue")
[ax[j].get_yaxis().set_visible(False) for j in range(3)]
[ax[j].get_xaxis().set_visible(False) for j in range(3)]

In [ ]:
fig, ax = plt.subplots()
base_offset = 2.58

num_images = 5
expected_min = 90
expected_max = 110

bin_img = np.zeros((480,640), "uint8")
# bin_img[152:202,350:389] =  1
bin_img[165:212,392:433] =  1

for i in range(1,num_images+1):
    imageToDisp = getImagePath(photo_width, photo_height, i)
    RGB_image = cv2.imread(imageToDisp)
    pair_img = cv2.imread(imageToDisp,0)
    imgLeft = pair_img[0:photo_height, 0:image_width]  # Y+H and X+W
    imgRight = pair_img[0:photo_height, image_width:photo_width]  # Y+H and X+W
    calibration = StereoCalibration(input_folder='calib_result2')
    rectified_pair = calibration.rectify((imgLeft, imgRight))
    dmLeft = rectified_pair[0]
    dmRight = rectified_pair[1]
    sbm = cv2.StereoBM_create(numDisparities=16, blockSize=15)
    disparity_pixels = sbm.compute(dmLeft, dmRight)
    imgLeft_RGB = RGB_image[0:photo_height, 0:image_width, :]

#     bin_img = generateBinaryImgStereo(imgLeft_RGB)
    visual = fire_disparity(bin_img, disparity)
    I = fire_disparity(bin_img, disparity_pixels)
    mean = np.sum(I)/np.count_nonzero(I) #avg disparity of fire pixels
    distance = distance_to_camera(mean)
    if distance > expected_min and distance < expected_max:
        color = "#d19fe8"
    elif distance < 0:
        color = "#D3D3D3"
        distance = 0
    else:
        color = '#c32148'
    print(distance)
    plt.scatter(i, distance, marker = 'o', c = color)

ax.plot(range(1,num_images+1), [(expected_min + expected_max)/2]*num_images, '--', linewidth=2, color='#d62728')
ax.set_ylabel("Distance (inches)")
ax.set_xlabel("Image #")
ax.set_title("Distance from Fire to Camera")
plt.show()

In [ ]:
fig, ax = plt.subplots(ncols = 2)
imageToDisp = getImagePath(photo_width, photo_height, 1)
RGB_image = cv2.imread(imageToDisp)
pair_img = cv2.imread(imageToDisp,0)
imgLeft = pair_img[0:photo_height, 0:image_width]  # Y+H and X+W
imgRight = pair_img[0:photo_height, image_width:photo_width]  # Y+H and X+W
calibration = StereoCalibration(input_folder='calib_result2')
rectified_pair = calibration.rectify((imgLeft, imgRight))
dmLeft = rectified_pair[0]
dmRight = rectified_pair[1]
sbm = cv2.StereoBM_create(numDisparities=16, blockSize=15)
disparity_pixels = sbm.compute(dmLeft, dmRight)
imgLeft_RGB = RGB_image[0:photo_height, 0:image_width, :]

# bin_img = generateBinaryImgStereo(imgLeft_RGB)

visual = fire_disparity(bin_img, disparity)
I = fire_disparity(bin_img, disparity_pixels)
mean = np.sum(I)/np.count_nonzero(I) #avg disparity of fire pixels
distance = distance_to_camera(mean)

visual = fire_disparity(bin_img, disparity)
ax[0].imshow(stereo_depth_map(rectified_pair),cmap='jet')  #cmap='jet'
ax[1].imshow(visual,cmap='jet')
print("mean disparity: ", mean)
print("distance: ", distance_to_camera(mean))

In [ ]:
print(np.mean(disparity_pixels))